**Imports**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from scipy.sparse import csr_matrix

In [3]:
from implicit import als

In [4]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [5]:
from src.recommenders_alt import alt_recommender

c:\users\пк\appdata\local\programs\python\python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [6]:
%load_ext autoreload

In [7]:
%autoreload

**Reading of data**

In [8]:
data = pd.read_csv('retail_train.csv')
data_test = pd.read_csv('retail_test.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

**Precessing of train data**

In [9]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

In [10]:
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

Т.к. есть отдельный валидационный датасет валидацию будем проводить на нем.

In [13]:
# val_lvl_1_size_weeks = 6
# val_lvl_2_size_weeks = 3

# data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
# data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
#                       (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]
data_train_lvl_1 = data.copy()
data_val_lvl_1 = data_test.copy()

In [11]:
data_test.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0


In [12]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


Датасеты для модели второго уровня.

In [ ]:
# data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
# data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

# data_train_lvl_1.head(2)

In [15]:
data_train_lvl_1.shape

(2396804, 12)

In [14]:
data_val_lvl_1.shape

(88734, 12)

Применим фильтры к датасетам

In [16]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 89051 to 5001


In [18]:
data_train_lvl_1.shape

(981281, 13)

In [19]:
n_items_before = data_val_lvl_1['item_id'].nunique()

data_val_lvl_1 = prefilter_items(data_val_lvl_1, take_n_popular=5000, item_features=item_features)

n_items_after = data_val_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 20497 to 5001


Обучим модель.

In [20]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

Готовим файл с результатами.  
По итогу выберем оптимальное значение с которым и будем работать в дальнейшем.

In [33]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[999999, 883616, 940947, 959219, 991024, 10049..."
1,2,"[999999, 866211, 879769, 885023, 899624, 94094..."


Recomendations for different types of recommenders

In [34]:
for k in [5]:
    print(f'Iteration k={k}')
    result_lvl_1[f'als_{k}'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=k))
    print(f'...als finished')
    result_lvl_1[f'own_{k}'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=k))
    print(f'...own finished')
    result_lvl_1[f'sim_user_{k}'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=k))
    print(f'...sim_user finished')
    result_lvl_1[f'sim_item_{k}'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=k))
    print(f'...sim_item finished')



Iteration k=5
...als finished
...own finished
...sim_user finished
...sim_item finished


In [35]:
result_lvl_1.head(2)

,user_id,actual,als_5,own_5,sim_user_5,sim_item_5
0,1,"[999999, 883616, 940947, 959219, 991024, 10049...","[1090247, 8293439, 6533362, 6534030, 8090541]","[856942, 9297615, 5577022, 8293439, 9655212]","[990869, 7160764, 982955, 847962, 1057168]","[1011089, 1007512, 9297615, 5577022, 825123]"
1,2,"[999999, 866211, 879769, 885023, 899624, 94094...","[1051211, 1106523, 1102067, 1114811, 1029743]","[1103898, 911974, 1076580, 5567582, 1007414]","[1028238, 6552318, 902377, 1131625, 903609]","[1044078, 8090537, 5569845, 985999, 819978]"


In [36]:
result_lvl_1.isna().sum()

user_id        0
actual         0
als_5          1
own_5          1
sim_user_5    14
sim_item_5     0
dtype: int64

In [37]:
result_lvl_1 = result_lvl_1.dropna()

Подсчет значений по метрике precision@k

In [39]:
total_result_lvl_1 = pd.DataFrame([{}])
for k in [5]:
    total_result_lvl_1[f'als_{k}'] = result_lvl_1.apply(lambda row: precision_at_k(row[f'als_{k}'], row['actual'], k), axis=1).mean()
    total_result_lvl_1[f'own_{k}'] = result_lvl_1.apply(lambda row: precision_at_k(row[f'own_{k}'], row['actual'], k), axis=1).mean()
    total_result_lvl_1[f'sim_user_{k}'] = result_lvl_1.apply(lambda row: precision_at_k(row[f'sim_user_{k}'], row['actual'], k), axis=1).mean()
    total_result_lvl_1[f'sim_item_{k}'] = result_lvl_1.apply(lambda row: precision_at_k(row[f'sim_item_{k}'], row['actual'], k), axis=1).mean()

In [40]:
total_result_lvl_1

,als_5,own_5,sim_user_5,sim_item_5
0,0.078959,0.120487,0.00155,0.054042
